In [1]:
import pandas as pd
from joblib import dump
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

In [2]:
df = pd.read_csv('divorce_data.csv', sep=';')
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Divorce
0,2,2,4,1,0,0,0,0,0,0,...,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,...,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,...,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,...,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,...,2,1,2,3,2,2,2,1,0,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 55 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Q1       170 non-null    int64
 1   Q2       170 non-null    int64
 2   Q3       170 non-null    int64
 3   Q4       170 non-null    int64
 4   Q5       170 non-null    int64
 5   Q6       170 non-null    int64
 6   Q7       170 non-null    int64
 7   Q8       170 non-null    int64
 8   Q9       170 non-null    int64
 9   Q10      170 non-null    int64
 10  Q11      170 non-null    int64
 11  Q12      170 non-null    int64
 12  Q13      170 non-null    int64
 13  Q14      170 non-null    int64
 14  Q15      170 non-null    int64
 15  Q16      170 non-null    int64
 16  Q17      170 non-null    int64
 17  Q18      170 non-null    int64
 18  Q19      170 non-null    int64
 19  Q20      170 non-null    int64
 20  Q21      170 non-null    int64
 21  Q22      170 non-null    int64
 22  Q23      170 non-null    i

In [4]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1:]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
classifier = SVC(kernel='rbf', degree=5, probability=True)
classifier.fit(x_train, y_train)
pred = classifier.predict_proba(x_test)
pred_c = [[value[1]>=0.5] for value in pred]
cm = confusion_matrix(y_test, pred_c)
cm

C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinsta

array([[14,  0],
       [ 0, 20]], dtype=int64)

In [6]:
pred

array([[0.95932894, 0.04067106],
       [0.00743682, 0.99256318],
       [0.97418312, 0.02581688],
       [0.00781277, 0.99218723],
       [0.9118708 , 0.0881292 ],
       [0.94119862, 0.05880138],
       [0.96815406, 0.03184594],
       [0.00535696, 0.99464304],
       [0.9756241 , 0.0243759 ],
       [0.00704414, 0.99295586],
       [0.00962773, 0.99037227],
       [0.93024385, 0.06975615],
       [0.97016334, 0.02983666],
       [0.00671347, 0.99328653],
       [0.00359066, 0.99640934],
       [0.99082228, 0.00917772],
       [0.01379177, 0.98620823],
       [0.98464485, 0.01535515],
       [0.03397013, 0.96602987],
       [0.96266923, 0.03733077],
       [0.00799861, 0.99200139],
       [0.00962773, 0.99037227],
       [0.00657708, 0.99342292],
       [0.00894812, 0.99105188],
       [0.01004651, 0.98995349],
       [0.41743888, 0.58256112],
       [0.00786305, 0.99213695],
       [0.9695646 , 0.0304354 ],
       [0.00741472, 0.99258528],
       [0.00667363, 0.99332637],
       [0.

In [10]:
# model: Scikit-Learn modeliniz
initial_type = [('float_input', FloatTensorType([None, len(x.columns)]))]
onnx_model = convert_sklearn(classifier, initial_types=initial_type)

with open("divorce_svc.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [12]:
x_test.iloc[0,:].values

array([3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 1, 1, 0,
       3, 3, 2, 2, 0, 2, 2, 0, 0, 4], dtype=int64)

In [13]:
len(x_test)

34